### Dataset Loading and and separates it into the training, validation, and testing sets
Note: Dataset source: https://huggingface.co/datasets/dair-ai/emotion

In [8]:
from datasets import load_dataset
dataset = load_dataset("dair-ai/emotion", "split")

train = dataset['train']
val = dataset['validation']
test = dataset['test']

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

### Separates training data into 2 arrays: “tweets” and “labels”.

In [10]:
def get_tweet(data):
    tweets = [x['text'] for x in data]
    labels = [x['label'] for x in data]
    return tweets, labels

tweets, labels = get_tweet(train)

tweets[:5], labels[:5]

(['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'im grabbing a minute to post i feel greedy wrong',
  'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
  'i am feeling grouchy'],
 [0, 0, 3, 2, 3])

### Initialize a tokenizer and calibrate it onto training data
This will assign each word a number by how commonly they appear in the dataset

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=10000, oov_token='<UNK>')
tokenizer.fit_on_texts(tweets)

tokenizer.texts_to_sequences([tweets[0]]) , tweets[0]

([[2, 139, 3, 679]], 'i didnt feel humiliated')

### Making all Sequences Same Shape

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

maxlen=50
def get_sequences(tokenizer, tweets):
    sequences = tokenizer.texts_to_sequences(tweets)
    padded = pad_sequences(sequences, truncating = 'post', padding='post', maxlen=maxlen)
    return padded

padded_train_seq = get_sequences(tokenizer, tweets)

padded_train_seq


array([[   2,  139,    3, ...,    0,    0,    0],
       [   2,   40,  101, ...,    0,    0,    0],
       [  17, 3060,    7, ...,    0,    0,    0],
       ...,
       [   2,    3,  327, ...,    0,    0,    0],
       [   2,    3,   14, ...,    0,    0,    0],
       [   2,   47,    7, ...,    0,    0,    0]], dtype=int32)

### Preparing Data for Model

In [13]:
import numpy as np

classes = set(labels)
class_to_index = dict((c,i) for i, c in enumerate(classes))
index_to_class = dict((v,k) for k, v in class_to_index.items())
names_to_ids = lambda labels: np.array([class_to_index.get(x) for x in labels])
train_labels = names_to_ids(labels)

print(classes)
print(class_to_index)
print(index_to_class)
print(train_labels)

{0, 1, 2, 3, 4, 5}
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}
[0 0 3 ... 1 3 0]


### Creating Model

In [14]:
from tensorflow import keras

model = keras.models.Sequential([
keras.layers.Embedding(10000,16),
keras.layers.Bidirectional(keras.layers.LSTM(20, return_sequences=True)),
keras.layers.Bidirectional(keras.layers.LSTM(20)),
keras.layers.Dense(6, activation='softmax')
])
model.compile(
     loss='sparse_categorical_crossentropy',
     optimizer='adam',
     metrics=['accuracy']
)

### Training Model

In [15]:
val_tweets, val_labels = get_tweet(val)
val_seq = get_sequences(tokenizer, val_tweets)
val_labels= names_to_ids(val_labels)
h = model.fit(
     padded_train_seq, train_labels,
     validation_data=(val_seq, val_labels),
     epochs=20,
     callbacks=[keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)]
)

Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 52s 80ms/step - accuracy: 0.3694 - loss: 1.5408 - val_accuracy: 0.5775 - val_loss: 1.1581
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 33s 65ms/step - accuracy: 0.6680 - loss: 0.9152 - val_accuracy: 0.7600 - val_loss: 0.6800
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 36s 72ms/step - accuracy: 0.8456 - loss: 0.4679 - val_accuracy: 0.8300 - val_loss: 0.5283
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 31s 63ms/step - accuracy: 0.9109 - loss: 0.2824 - val_accuracy: 0.8570 - val_loss: 0.4439
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 38s 75ms/step - accuracy: 0.9406 - loss: 0.2121 - val_accuracy: 0.8605 - val_loss: 0.4842
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 32s 57ms/step - accuracy: 0.9554 - loss: 0.1556 - val_accuracy: 0.8730 - val_loss: 0.4582
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 24s 49ms/step - accuracy: 0.9622 - loss: 0.1286 - val_accuracy: 0.8880 - val_loss: 0.4003
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 28s 56ms/step - accuracy: 0.9744 - loss: 0.0935 - 

### Evaluating and Testing Model

In [16]:
test_tweets, test_labels=get_tweet(test)
test_seq = get_sequences(tokenizer, test_tweets)
test_labels=names_to_ids(test_labels)
model.evaluate(test_seq, test_labels)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8698 - loss: 0.4334


[0.43199121952056885, 0.8730000257492065]

In [17]:
import random

i = random.randint(0,len(test_labels)-1)
print('Sentence:', test_tweets[i])
print('Emotion:', index_to_class[test_labels[i]])
p = model.predict(np.expand_dims(test_seq[i], axis=0))[0]
print(test_seq[i])
pred_class=index_to_class[np.argmax(p).astype('uint8')]
print('Predicted Emotion: ', pred_class)

Sentence: i feel ugly and hated
Emotion: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 747ms/step
[  2   3 585   4 303   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
Predicted Emotion:  0


In [18]:
sentence = 'i am happy'
sequence = tokenizer.texts_to_sequences([sentence])
paddedSequence = pad_sequences(sequence, truncating = 'post', padding='post', maxlen=maxlen)
p = model.predict(np.expand_dims(paddedSequence[0], axis=0))[0]
pred_class=index_to_class[np.argmax(p).astype('uint8')]
print('Sentence:', sentence)
print('Predicted Emotion: ', pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Sentence: i am happy
Predicted Emotion:  1
